# SAS Project

## Imports

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import sklearn as skl
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA
from scipy.stats import norm
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.metrics import calinski_harabasz_score
from sklearn.metrics import silhouette_score
from scipy.spatial.distance import cdist

import datetime
from datetime import datetime
import networkx as nx
import pickle
import csv
from multiprocessing import Pool
from collections import ChainMap
from scipy.special import logit, expit


import warnings
warnings.filterwarnings("ignore")


import lightgbm
from lightgbm import LGBMRegressor
from sklearn.model_selection import RandomizedSearchCV

import joblib
import os
from pathlib import Path

from catboost import CatBoostClassifier
import math

from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.layers import Dense, LSTM

pd.set_option("display.max_columns", None)

2022-06-18 12:44:12.328755: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-18 12:44:12.328796: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Preprocessing and Scaling

In [2]:
loc = pd.read_csv("LOCATION.csv", delimiter = ';')
loc= loc[['STORE_LOCATION_RK', 'STORE_LOCATION_LVL_RK4','STORE_LOCATION_LVL_RK3',
          'STORE_LOCATION_LVL_RK2',	'STORE_LOCATION_LVL_RK1',	'STORE_OPEN_DTTM',
          'STORE_CLOSURE_DTTM']]
          # all of "hash" data is in an unknown format w/o description => drop
loc = loc.rename(columns = {'STORE_LOCATION_RK' : 'location_id', 
                      'STORE_LOCATION_LVL_RK4' : 'STORE_LOCATION_LVL4', 
                      'STORE_LOCATION_LVL_RK3' : 'STORE_LOCATION_LVL3',
                      'STORE_LOCATION_LVL_RK2' : 'STORE_LOCATION_LVL2', 
                      'STORE_LOCATION_LVL_RK1' : 'STORE_LOCATION_LVL1' 
                      })
#['STORE_OPEN'].isnull().sum() - Open&Closure are empty => abs uninformative => drop
loc = loc.drop(columns = ['STORE_OPEN_DTTM', 'STORE_CLOSURE_DTTM'])
# small to none unqiue value & non-informative
loc = loc.drop(columns = ['STORE_LOCATION_LVL1', 'STORE_LOCATION_LVL2'])
loc

,location_id,STORE_LOCATION_LVL4,STORE_LOCATION_LVL3
0,277,124,124
1,278,124,124
2,279,183,183
3,280,211,211
4,281,221,221
...,...,...,...
1203,1482,227,227
1204,1483,124,124
1205,1484,24,24
1206,1485,124,124


In [3]:
prod = pd.read_csv('PRODUCT.csv', delimiter = ';')
prod= prod[['PRODUCT_RK', 'PRODUCT_LVL_RK6',
          'PRODUCT_LVL_RK5','PRODUCT_LVL_RK4', 'PRODUCT_LVL_RK3','PRODUCT_LVL_RK2',
          'SALES_INTRODUCTION_DT',	
          'SALES_DISCONTINUED_DT']]
          # all of "hash" data is in an unknown format w/o description => drop
prod = prod.rename(columns = {'STORE_LOCATION_RK' : 'STORE_LOCATION', 
                      'PRODUCT_RK' : 'product_id',
                      'PRODUCT_LVL_RK6' : 'PRODUCT_LVL6',
                      'PRODUCT_LVL_RK5' : 'PRODUCT_LVL5',
                      'PRODUCT_LVL_RK4' : 'PRODUCT_LVL4',
                      'PRODUCT_LVL_RK3' : 'PRODUCT_LVL3',
                      'PRODUCT_LVL_RK2' : 'PRODUCT_LVL2',
                      'SALES_INTRODUCTION_DT' : 'SALES_INTRODUCTION', 
                      'SALES_DISCONTINUED_DT' : 'SALES_DISCONTINUED'})
# prod['SALES_INTRODUCTION'].isnull().sum()- SalesIntro&Discontinue are empty => abs uninformative => drop 
prod = prod.drop(columns = ['SALES_INTRODUCTION', 'SALES_DISCONTINUED'])
# small to none unqiue value & non-informative
prod = prod.drop(columns = ['PRODUCT_LVL2', 'PRODUCT_LVL3'])
prod

,product_id,PRODUCT_LVL6,PRODUCT_LVL5,PRODUCT_LVL4
0,11316,10224,10198,113533
1,11317,10224,10198,113533
2,11318,10224,10198,113533
3,11319,10224,10198,113533
4,11320,10224,10198,113533
...,...,...,...,...
171603,191028,190706,6196,113158
171604,191029,73057,113184,6204
171605,191030,73057,113184,6204
171606,191031,73057,113184,6204


Get training data

In [4]:
train = pd.read_csv('lt_train.csv')
train

,period_dt,location_id,product_id,demand,PROMO1_FLAG,PROMO2_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,NUM_CONSULTANT,AUTORIZATION_FLAG,id
0,2019-12-16,764.0,23285.0,NaN,0.0,0.0,2199.0,1099.5,0.0,1.0,1
1,2019-12-30,764.0,23285.0,NaN,0.0,0.0,2199.0,1099.5,0.0,1.0,3
2,2019-12-09,764.0,23285.0,NaN,0.0,0.0,2199.0,1099.5,0.0,0.0,4
3,2019-12-02,453.0,23285.0,NaN,1.0,0.0,2199.0,1319.4,0.0,1.0,5
4,2019-12-02,764.0,23285.0,NaN,1.0,0.0,2199.0,1099.5,0.0,0.0,6
...,...,...,...,...,...,...,...,...,...,...,...
3404841,2019-12-30,NaN,NaN,NaN,1.0,0.0,NaN,NaN,0.0,0.0,3409994
3404842,2019-12-30,NaN,NaN,NaN,1.0,0.0,NaN,NaN,0.0,0.0,3409995
3404843,2019-12-30,NaN,NaN,NaN,1.0,0.0,NaN,NaN,0.0,0.0,3409996
3404844,2019-12-30,NaN,NaN,NaN,1.0,0.0,NaN,NaN,0.0,0.0,3409997


Only 10% of the data has something informative stats with regards to price

In [5]:
train['demand'].isnull().sum() 

3155253

The price is highly ditributed with lots of outliers

In [ ]:
fig = px.histogram(train, x = 'PRICE_REGULAR')
fig.show()

In [7]:
train['PRICE_REGULAR'].describe() 

count    2.182591e+06
mean     2.256149e+03
std      5.514987e+02
min      4.950000e+02
25%      1.999000e+03
50%      2.399000e+03
75%      2.600000e+03
max      5.699000e+03
Name: PRICE_REGULAR, dtype: float64

Drop entries with no data with regards to products

In [8]:
train['product_id'].isnull().sum()
train = train.dropna(subset = ['product_id']).reset_index(drop=True)
train

,period_dt,location_id,product_id,demand,PROMO1_FLAG,PROMO2_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,NUM_CONSULTANT,AUTORIZATION_FLAG,id
0,2019-12-16,764.0,23285.0,NaN,0.0,0.0,2199.0,1099.50,0.0,1.0,1
1,2019-12-30,764.0,23285.0,NaN,0.0,0.0,2199.0,1099.50,0.0,1.0,3
2,2019-12-09,764.0,23285.0,NaN,0.0,0.0,2199.0,1099.50,0.0,0.0,4
3,2019-12-02,453.0,23285.0,NaN,1.0,0.0,2199.0,1319.40,0.0,1.0,5
4,2019-12-02,764.0,23285.0,NaN,1.0,0.0,2199.0,1099.50,0.0,0.0,6
...,...,...,...,...,...,...,...,...,...,...,...
402897,2019-11-11,1316.0,135868.0,NaN,1.0,0.0,NaN,NaN,5.0,1.0,408050
402898,2019-11-11,1316.0,135869.0,NaN,1.0,0.0,NaN,NaN,5.0,1.0,408051
402899,2019-11-11,1316.0,149418.0,NaN,1.0,0.0,2949.0,1474.50,0.0,1.0,408052
402900,2019-11-25,453.0,40608.0,NaN,1.0,0.0,1549.0,464.70,0.0,0.0,408053


Merge info about locations and products with training dataset

In [9]:
df = train.merge(prod, how = 'left', on = 'product_id')
df = df.merge(loc, how = 'left', on = 'location_id')
df

,period_dt,location_id,product_id,demand,PROMO1_FLAG,PROMO2_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,NUM_CONSULTANT,AUTORIZATION_FLAG,id,PRODUCT_LVL6,PRODUCT_LVL5,PRODUCT_LVL4,STORE_LOCATION_LVL4,STORE_LOCATION_LVL3
0,2019-12-16,764.0,23285.0,NaN,0.0,0.0,2199.0,1099.50,0.0,1.0,1,3834,6152,3832,219,219
1,2019-12-30,764.0,23285.0,NaN,0.0,0.0,2199.0,1099.50,0.0,1.0,3,3834,6152,3832,219,219
2,2019-12-09,764.0,23285.0,NaN,0.0,0.0,2199.0,1099.50,0.0,0.0,4,3834,6152,3832,219,219
3,2019-12-02,453.0,23285.0,NaN,1.0,0.0,2199.0,1319.40,0.0,1.0,5,3834,6152,3832,189,189
4,2019-12-02,764.0,23285.0,NaN,1.0,0.0,2199.0,1099.50,0.0,0.0,6,3834,6152,3832,219,219
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402897,2019-11-11,1316.0,135868.0,NaN,1.0,0.0,NaN,NaN,5.0,1.0,408050,3834,6152,3832,203,203
402898,2019-11-11,1316.0,135869.0,NaN,1.0,0.0,NaN,NaN,5.0,1.0,408051,3834,6152,3832,203,203
402899,2019-11-11,1316.0,149418.0,NaN,1.0,0.0,2949.0,1474.50,0.0,1.0,408052,3839,6152,3832,203,203
402900,2019-11-25,453.0,40608.0,NaN,1.0,0.0,1549.0,464.70,0.0,0.0,408053,3834,6152,3832,189,189


Extract day/week/month/year

In [10]:
df['period_dt'] = pd.to_datetime(df['period_dt'])
df['year'] = pd.to_datetime(df['period_dt']).dt.year
df['month'] = pd.to_datetime(df['period_dt']).dt.month
df['day'] = pd.to_datetime(df['period_dt']).dt.day
df['week'] = pd.to_datetime(df['period_dt']).dt.week
df = df.drop('period_dt', axis=1)
df

,location_id,product_id,demand,PROMO1_FLAG,PROMO2_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,NUM_CONSULTANT,AUTORIZATION_FLAG,id,PRODUCT_LVL6,PRODUCT_LVL5,PRODUCT_LVL4,STORE_LOCATION_LVL4,STORE_LOCATION_LVL3,year,month,day,week
0,764.0,23285.0,NaN,0.0,0.0,2199.0,1099.50,0.0,1.0,1,3834,6152,3832,219,219,2019,12,16,51
1,764.0,23285.0,NaN,0.0,0.0,2199.0,1099.50,0.0,1.0,3,3834,6152,3832,219,219,2019,12,30,1
2,764.0,23285.0,NaN,0.0,0.0,2199.0,1099.50,0.0,0.0,4,3834,6152,3832,219,219,2019,12,9,50
3,453.0,23285.0,NaN,1.0,0.0,2199.0,1319.40,0.0,1.0,5,3834,6152,3832,189,189,2019,12,2,49
4,764.0,23285.0,NaN,1.0,0.0,2199.0,1099.50,0.0,0.0,6,3834,6152,3832,219,219,2019,12,2,49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402897,1316.0,135868.0,NaN,1.0,0.0,NaN,NaN,5.0,1.0,408050,3834,6152,3832,203,203,2019,11,11,46
402898,1316.0,135869.0,NaN,1.0,0.0,NaN,NaN,5.0,1.0,408051,3834,6152,3832,203,203,2019,11,11,46
402899,1316.0,149418.0,NaN,1.0,0.0,2949.0,1474.50,0.0,1.0,408052,3839,6152,3832,203,203,2019,11,11,46
402900,453.0,40608.0,NaN,1.0,0.0,1549.0,464.70,0.0,0.0,408053,3834,6152,3832,189,189,2019,11,25,48


Use robust scaler for ditributed variables to properly scale the data with outliers

In [11]:
labels = ['PRICE_REGULAR', 'PRICE_AFTER_DISC', 'NUM_CONSULTANT','PRODUCT_LVL6', 
          'PRODUCT_LVL5', 'PRODUCT_LVL4', 'STORE_LOCATION_LVL4', 'STORE_LOCATION_LVL3']


ptdf = df[labels]
ptdf = RobustScaler().fit_transform(ptdf)
ptdf = pd.DataFrame(data = ptdf, columns = labels)
df[labels] = ptdf.copy()
df

,location_id,product_id,demand,PROMO1_FLAG,PROMO2_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,NUM_CONSULTANT,AUTORIZATION_FLAG,id,PRODUCT_LVL6,PRODUCT_LVL5,PRODUCT_LVL4,STORE_LOCATION_LVL4,STORE_LOCATION_LVL3,year,month,day,week
0,764.0,23285.0,NaN,0.0,0.0,-0.248297,-0.637029,0.0,1.0,1,-5.0,0.0,0.0,0.201342,0.201342,2019,12,16,51
1,764.0,23285.0,NaN,0.0,0.0,-0.248297,-0.637029,0.0,1.0,3,-5.0,0.0,0.0,0.201342,0.201342,2019,12,30,1
2,764.0,23285.0,NaN,0.0,0.0,-0.248297,-0.637029,0.0,0.0,4,-5.0,0.0,0.0,0.201342,0.201342,2019,12,9,50
3,453.0,23285.0,NaN,1.0,0.0,-0.248297,-0.240755,0.0,1.0,5,-5.0,0.0,0.0,0.000000,0.000000,2019,12,2,49
4,764.0,23285.0,NaN,1.0,0.0,-0.248297,-0.637029,0.0,0.0,6,-5.0,0.0,0.0,0.201342,0.201342,2019,12,2,49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402897,1316.0,135868.0,NaN,1.0,0.0,NaN,NaN,5.0,1.0,408050,-5.0,0.0,0.0,0.093960,0.093960,2019,11,11,46
402898,1316.0,135869.0,NaN,1.0,0.0,NaN,NaN,5.0,1.0,408051,-5.0,0.0,0.0,0.093960,0.093960,2019,11,11,46
402899,1316.0,149418.0,NaN,1.0,0.0,0.984966,0.038744,0.0,1.0,408052,0.0,0.0,0.0,0.093960,0.093960,2019,11,11,46
402900,453.0,40608.0,NaN,1.0,0.0,-1.317125,-1.780977,0.0,0.0,408053,-5.0,0.0,0.0,0.000000,0.000000,2019,11,25,48


Save the preprocessed data for further tries of training

In [12]:
df['location_id'] = df['location_id'].astype(int)
df['product_id'] = df['product_id'].astype(int)
df.to_csv('sas_dataframe_scaled.csv') 

## Modelling

In [13]:
del df, ptdf, train, prod

In [14]:
init = pd.read_csv('sas_dataframe_scaled.csv', index_col=0)
df = init.copy()
df

,location_id,product_id,demand,PROMO1_FLAG,PROMO2_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,NUM_CONSULTANT,AUTORIZATION_FLAG,id,PRODUCT_LVL6,PRODUCT_LVL5,PRODUCT_LVL4,STORE_LOCATION_LVL4,STORE_LOCATION_LVL3,year,month,day,week
0,764,23285,NaN,0.0,0.0,-0.248297,-0.637029,0.0,1.0,1,-5.0,0.0,0.0,0.201342,0.201342,2019,12,16,51
1,764,23285,NaN,0.0,0.0,-0.248297,-0.637029,0.0,1.0,3,-5.0,0.0,0.0,0.201342,0.201342,2019,12,30,1
2,764,23285,NaN,0.0,0.0,-0.248297,-0.637029,0.0,0.0,4,-5.0,0.0,0.0,0.201342,0.201342,2019,12,9,50
3,453,23285,NaN,1.0,0.0,-0.248297,-0.240755,0.0,1.0,5,-5.0,0.0,0.0,0.000000,0.000000,2019,12,2,49
4,764,23285,NaN,1.0,0.0,-0.248297,-0.637029,0.0,0.0,6,-5.0,0.0,0.0,0.201342,0.201342,2019,12,2,49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402897,1316,135868,NaN,1.0,0.0,NaN,NaN,5.0,1.0,408050,-5.0,0.0,0.0,0.093960,0.093960,2019,11,11,46
402898,1316,135869,NaN,1.0,0.0,NaN,NaN,5.0,1.0,408051,-5.0,0.0,0.0,0.093960,0.093960,2019,11,11,46
402899,1316,149418,NaN,1.0,0.0,0.984966,0.038744,0.0,1.0,408052,0.0,0.0,0.0,0.093960,0.093960,2019,11,11,46
402900,453,40608,NaN,1.0,0.0,-1.317125,-1.780977,0.0,0.0,408053,-5.0,0.0,0.0,0.000000,0.000000,2019,11,25,48


Drop entries with no data

In [15]:
df = df.dropna().reset_index(drop=True)
df['PROMO1_FLAG'] = df['PROMO1_FLAG'].astype(int)
df['PROMO2_FLAG'] = df['PROMO2_FLAG'].astype(int)
df['NUM_CONSULTANT'] = df['NUM_CONSULTANT'].astype(int)
df['AUTORIZATION_FLAG'] = df['AUTORIZATION_FLAG'].astype(int)
df

,location_id,product_id,demand,PROMO1_FLAG,PROMO2_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,NUM_CONSULTANT,AUTORIZATION_FLAG,id,PRODUCT_LVL6,PRODUCT_LVL5,PRODUCT_LVL4,STORE_LOCATION_LVL4,STORE_LOCATION_LVL3,year,month,day,week
0,355,23285,0.002740,1,0,-0.412732,0.596753,0,1,7,-5.0,0.0,0.0,-0.90604,-0.90604,2017,1,2,1
1,355,23285,1.000000,1,0,-0.248297,0.353655,0,1,8,-5.0,0.0,0.0,-0.90604,-0.90604,2017,2,27,9
2,355,23285,1.016438,0,0,-0.248297,0.353655,0,1,9,-5.0,0.0,0.0,-0.90604,-0.90604,2017,5,22,21
3,355,23285,0.038356,0,0,-0.248297,0.353655,0,1,10,-5.0,0.0,0.0,-0.90604,-0.90604,2017,5,29,22
4,355,23285,0.038356,0,0,-0.248297,0.353655,0,1,11,-5.0,0.0,0.0,-0.90604,-0.90604,2017,6,5,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242796,1363,119952,2.000000,1,0,3.040404,4.948461,0,0,407298,0.0,0.0,0.0,-1.09396,-1.09396,2018,10,1,40
242797,1363,119952,1.190476,0,0,3.040404,3.597237,0,0,407299,0.0,0.0,0.0,-1.09396,-1.09396,2018,10,22,43
242798,1363,119952,0.285714,1,0,3.040404,3.056747,0,0,407300,0.0,0.0,0.0,-1.09396,-1.09396,2018,10,29,44
242799,1363,119952,0.190476,1,0,3.040404,3.056747,0,0,407301,0.0,0.0,0.0,-1.09396,-1.09396,2018,11,12,46


Pull data to test from train

In [16]:
df_1 = pd.read_csv('lt_test.csv')
df_1 = df_1.drop(columns=['product_id', 'location_id', 'period_dt'])
pdf = init.merge(df_1, how='inner', on='id')
pdf = pdf.drop(columns=['demand'])
pdf

,location_id,product_id,PROMO1_FLAG,PROMO2_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,NUM_CONSULTANT,AUTORIZATION_FLAG,id,PRODUCT_LVL6,PRODUCT_LVL5,PRODUCT_LVL4,STORE_LOCATION_LVL4,STORE_LOCATION_LVL3,year,month,day,week
0,317,23297,1.0,0.0,-0.330514,-0.294817,0.0,1.0,601,-5.0,0.0,0.0,0.00000,0.00000,2019,1,21,4
1,644,23297,1.0,0.0,-0.330514,-0.294817,0.0,1.0,697,-5.0,0.0,0.0,0.00000,0.00000,2019,1,28,5
2,1162,23297,1.0,0.0,-0.330514,-0.294817,0.0,1.0,831,-5.0,0.0,0.0,0.00000,0.00000,2019,1,14,3
3,1316,23297,0.0,0.0,-0.330514,-0.101186,0.0,1.0,875,-5.0,0.0,0.0,0.09396,0.09396,2019,2,25,9
4,644,23301,1.0,0.0,-0.659385,-0.511065,0.0,1.0,975,-5.0,0.0,0.0,0.00000,0.00000,2019,1,28,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10164,1316,135868,1.0,0.0,NaN,NaN,5.0,1.0,408050,-5.0,0.0,0.0,0.09396,0.09396,2019,11,11,46
10165,1316,135869,1.0,0.0,NaN,NaN,5.0,1.0,408051,-5.0,0.0,0.0,0.09396,0.09396,2019,11,11,46
10166,1316,149418,1.0,0.0,0.984966,0.038744,0.0,1.0,408052,0.0,0.0,0.0,0.09396,0.09396,2019,11,11,46
10167,453,40608,1.0,0.0,-1.317125,-1.780977,0.0,0.0,408053,-5.0,0.0,0.0,0.00000,0.00000,2019,11,25,48


Full NaNs in prices in test with means from stores

In [17]:
loc_feat = df.groupby(by='location_id').mean()[['demand', 'PRICE_REGULAR', 'PRICE_AFTER_DISC']]
loc_feat['location_id'] = loc_feat.index
loc_feat.columns = ['loc_demand', 'loc_pr', 'loc_ad', 'location_id']
loc_feat.index.name = None

df = df.merge(loc_feat, how='left', on='location_id')
pdf_1 = pdf.merge(loc_feat, how='left', on='location_id')
pdf_1['PRICE_REGULAR'] = pdf_1.apply(lambda x: x['loc_pr'] if np.isnan(x['PRICE_REGULAR']) else x['PRICE_REGULAR'], axis=1)
pdf_1['PRICE_AFTER_DISC'] = pdf_1.apply(lambda x: x['loc_ad'] if np.isnan(x['PRICE_AFTER_DISC']) else x['PRICE_AFTER_DISC'], axis=1)
pdf = pdf_1.copy()
pdf.index = pdf['id']
pdf

,location_id,product_id,PROMO1_FLAG,PROMO2_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,NUM_CONSULTANT,AUTORIZATION_FLAG,id,PRODUCT_LVL6,PRODUCT_LVL5,PRODUCT_LVL4,STORE_LOCATION_LVL4,STORE_LOCATION_LVL3,year,month,day,week,loc_demand,loc_pr,loc_ad
id,,,,,,,,,,,,,,,,,,,,,
601,317,23297,1.0,0.0,-0.330514,-0.294817,0.0,1.0,601,-5.0,0.0,0.0,0.00000,0.00000,2019,1,21,4,0.225602,-0.299622,0.104953
697,644,23297,1.0,0.0,-0.330514,-0.294817,0.0,1.0,697,-5.0,0.0,0.0,0.00000,0.00000,2019,1,28,5,0.207831,-0.404527,0.019090
831,1162,23297,1.0,0.0,-0.330514,-0.294817,0.0,1.0,831,-5.0,0.0,0.0,0.00000,0.00000,2019,1,14,3,0.317820,-0.237853,0.148540
875,1316,23297,0.0,0.0,-0.330514,-0.101186,0.0,1.0,875,-5.0,0.0,0.0,0.09396,0.09396,2019,2,25,9,1.028000,-0.013380,0.105102
975,644,23301,1.0,0.0,-0.659385,-0.511065,0.0,1.0,975,-5.0,0.0,0.0,0.00000,0.00000,2019,1,28,5,0.207831,-0.404527,0.019090
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
408050,1316,135868,1.0,0.0,-0.013380,0.105102,5.0,1.0,408050,-5.0,0.0,0.0,0.09396,0.09396,2019,11,11,46,1.028000,-0.013380,0.105102
408051,1316,135869,1.0,0.0,-0.013380,0.105102,5.0,1.0,408051,-5.0,0.0,0.0,0.09396,0.09396,2019,11,11,46,1.028000,-0.013380,0.105102
408052,1316,149418,1.0,0.0,0.984966,0.038744,0.0,1.0,408052,0.0,0.0,0.0,0.09396,0.09396,2019,11,11,46,1.028000,-0.013380,0.105102


Fill NaNa with zeroes otherwise

In [18]:
pdf['PROMO1_FLAG'] = pdf['PROMO1_FLAG'].apply(lambda x: 0 if np.isnan(x) else x)
pdf['PROMO2_FLAG'] = pdf['PROMO2_FLAG'].apply(lambda x: 0 if np.isnan(x) else x)
pdf['AUTORIZATION_FLAG'] = pdf['AUTORIZATION_FLAG'].apply(lambda x: 0 if np.isnan(x) else x)
pdf['NUM_CONSULTANT'] = pdf['NUM_CONSULTANT'].apply(lambda x: 0 if np.isnan(x) else x)

pdf['PROMO1_FLAG'] = pdf['PROMO1_FLAG'].astype(int)
pdf['PROMO2_FLAG'] = pdf['PROMO2_FLAG'].astype(int)
pdf['NUM_CONSULTANT'] = pdf['NUM_CONSULTANT'].astype(int)
pdf['AUTORIZATION_FLAG'] = pdf['AUTORIZATION_FLAG'].astype(int)

Use LSTM for prediction

In [19]:
from keras import backend as K

def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 


def LSTM_funct(train_data, test_data):
 
    x_train = train_data.drop('demand', axis=1)
    y_train = train_data['demand']
 
    model = Sequential()
    model.add(LSTM(50,return_sequences=True, input_shape = (x_train.shape[1], 1)))
    model.add(LSTM(50, return_sequences=False))
    model.add(Dense(25))
    model.add(Dense(1))
 
    stop = EarlyStopping(monitor='loss',min_delta=0.000000000001,patience=30) 
 
    model.compile(optimizer='nadam', loss='mean_absolute_error')
    model.fit(x_train, y_train, batch_size = 6000, epochs = 5, verbose=1)

    predictions = model.predict(test_data)
    return predictions, model


mpdf = pdf.copy()
pred, mdl = LSTM_funct(df,mpdf)

2022-06-18 12:44:59.167743: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-06-18 12:44:59.167795: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-18 12:44:59.167836: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (icar.sas.yp-c.yandex.net): /proc/driver/nvidia/version does not exist


Epoch 1/5
41/41 [==============================] - 25s 462ms/step - loss: 0.3557
Epoch 2/5
41/41 [==============================] - 17s 414ms/step - loss: 0.3411
Epoch 3/5
41/41 [==============================] - 17s 411ms/step - loss: 0.3392
Epoch 4/5
41/41 [==============================] - 17s 410ms/step - loss: 0.3395
Epoch 5/5
318/318 [==============================] - 5s 13ms/step


Save predictions

In [20]:
pdf['demand'] = pred
pdf['demand'].to_csv('LSTM_nAdam.csv')
pdf

,location_id,product_id,PROMO1_FLAG,PROMO2_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,NUM_CONSULTANT,AUTORIZATION_FLAG,id,PRODUCT_LVL6,PRODUCT_LVL5,PRODUCT_LVL4,STORE_LOCATION_LVL4,STORE_LOCATION_LVL3,year,month,day,week,loc_demand,loc_pr,loc_ad,demand
id,,,,,,,,,,,,,,,,,,,,,,
601,317,23297,1,0,-0.330514,-0.294817,0,1,601,-5.0,0.0,0.0,0.00000,0.00000,2019,1,21,4,0.225602,-0.299622,0.104953,0.058387
697,644,23297,1,0,-0.330514,-0.294817,0,1,697,-5.0,0.0,0.0,0.00000,0.00000,2019,1,28,5,0.207831,-0.404527,0.019090,0.066402
831,1162,23297,1,0,-0.330514,-0.294817,0,1,831,-5.0,0.0,0.0,0.00000,0.00000,2019,1,14,3,0.317820,-0.237853,0.148540,0.055669
875,1316,23297,0,0,-0.330514,-0.101186,0,1,875,-5.0,0.0,0.0,0.09396,0.09396,2019,2,25,9,1.028000,-0.013380,0.105102,0.076993
975,644,23301,1,0,-0.659385,-0.511065,0,1,975,-5.0,0.0,0.0,0.00000,0.00000,2019,1,28,5,0.207831,-0.404527,0.019090,0.065266
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
408050,1316,135868,1,0,-0.013380,0.105102,5,1,408050,-5.0,0.0,0.0,0.09396,0.09396,2019,11,11,46,1.028000,-0.013380,0.105102,0.156110
408051,1316,135869,1,0,-0.013380,0.105102,5,1,408051,-5.0,0.0,0.0,0.09396,0.09396,2019,11,11,46,1.028000,-0.013380,0.105102,0.156110
408052,1316,149418,1,0,0.984966,0.038744,0,1,408052,0.0,0.0,0.0,0.09396,0.09396,2019,11,11,46,1.028000,-0.013380,0.105102,0.177556
